In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('/kaggle/input/google-competition/train.csv')
data_test=pd.read_csv('/kaggle/input/google-competition/test.csv')
data.head()
data.dtypes
features=data.drop('exam_score', axis=1)
target=data.exam_score



In [2]:
data_test.head() 

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
0,630000,24,other,ba,6.85,65.2,yes,5.2,poor,group study,high,easy
1,630001,18,male,diploma,6.61,45.0,no,9.3,poor,coaching,low,easy
2,630002,24,female,b.tech,6.60,98.5,yes,6.2,good,group study,medium,moderate
3,630003,24,male,diploma,3.03,66.3,yes,5.7,average,mixed,medium,moderate
4,630004,20,female,b.tech,2.03,42.4,yes,9.2,average,coaching,low,moderate


In [3]:

data.dtypes

id                    int64
age                   int64
gender               object
course               object
study_hours         float64
class_attendance    float64
internet_access      object
sleep_hours         float64
sleep_quality        object
study_method         object
facility_rating      object
exam_difficulty      object
exam_score          float64
dtype: object

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

def k_fold_target_encode_split(train_df, test_df, cols, target, n_folds=5, smooth=10):
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    global_mean = train_df[target].mean()
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    for col in cols:
        new_col_name = f"{col}_encoded"
        
        # --- 1. HANDLE TRAINING DATA (K-FOLD) ---
        train_encoded[new_col_name] = np.nan
        
        for train_idx, val_idx in kf.split(train_df):
            # Use out-of-fold data to calculate means
            df_in = train_df.iloc[train_idx]
            
            agg = df_in.groupby(col)[target].agg(['count', 'mean'])
            counts = agg['count']
            means = agg['mean']
            
            # Apply Smoothing
            smooth_mean = (counts * means + smooth * global_mean) / (counts + smooth)
            
            # Map to the validation slice of the training set
            train_encoded.iloc[val_idx, train_encoded.columns.get_loc(new_col_name)] = \
                train_df.iloc[val_idx][col].map(smooth_mean)
        
        # Fill any new categories in training with global mean
        train_encoded[new_col_name] = train_encoded[new_col_name].fillna(global_mean)

        # --- 2. HANDLE TEST DATA (GLOBAL MEAN) ---
        # For test data, we use the full training set to build the mapping
        agg_full = train_df.groupby(col)[target].agg(['count', 'mean'])
        full_counts = agg_full['count']
        full_means = agg_full['mean']
        
        full_smooth_mean = (full_counts * full_means + smooth * global_mean) / (full_counts + smooth)
        
        test_encoded[new_col_name] = test_df[col].map(full_smooth_mean).fillna(global_mean)
    train_encoded = train_encoded.drop(columns=cols)
    test_encoded = test_encoded.drop(columns=cols)
                
    return train_encoded, test_encoded

# Example usage:
# df_train_enc, df_test_enc = k_fold_target_encode_split(train_df, test_df, ['City'], 'Target')

In [5]:
encode=['gender','course','internet_access','sleep_quality','study_method','facility_rating','exam_difficulty']

In [6]:
df_train_enc,df_test_enc=k_fold_target_encode_split(data,data_test,encode,'exam_score')

In [7]:

enc_features=df_train_enc.drop('exam_score',axis=1)
target=df_train_enc.exam_score

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
train_enc_features,val_enc_features,train_target,val_target=train_test_split(enc_features,target,random_state=0)
model=DecisionTreeRegressor()
model.fit(train_enc_features,train_target)
val_pred=model.predict(val_enc_features)

In [9]:
from sklearn.metrics import mean_absolute_error
val_mae=mean_absolute_error(val_target,val_pred)
print(val_mae)

10.167279879365077


In [10]:


model=DecisionTreeRegressor()
model.fit(enc_features,target)

df_test_enc['exam_score']=model.predict(df_test_enc)
df_test_enc.head()




,id,age,study_hours,class_attendance,sleep_hours,gender_encoded,course_encoded,internet_access_encoded,sleep_quality_encoded,study_method_encoded,facility_rating_encoded,exam_difficulty_encoded,exam_score
0,630000,24,6.85,65.2,5.2,62.781178,61.887031,62.509172,56.995728,60.531998,66.706919,62.214417,62.0
1,630001,18,6.61,45.0,9.3,62.175577,62.404374,62.478043,56.995728,69.266152,57.954760,62.214417,77.7
2,630002,24,6.60,98.5,6.2,62.563217,62.519706,62.509172,67.884633,60.531998,63.028929,62.606058,86.2
3,630003,24,3.03,66.3,5.7,62.175577,62.404374,62.509172,62.661992,65.101524,63.028929,62.606058,75.0
4,630004,20,2.03,42.4,9.2,62.563217,62.519706,62.509172,62.661992,69.266152,57.954760,62.606058,56.2
